# Computer Simulation Project
Mohammad Javad Maheronnaghsh

Mohammad Mowlavi

## General Information of the Simulation

In [6]:
# Imported Libraries
import numpy as np
import random

In [7]:


class Task:
    def __init__(self, inter_arrival=None, priority=None, execute_time=None):
        self.inter_arrival = inter_arrival
        self.priority = priority
        self.execute_time = execute_time


class BaseQueue:
    def __init__(self):
        pass

    def get_first(self):
        raise NotImplemented

    def add_arrival(self, task):
        raise NotImplemented


class FIFO(BaseQueue):

    def __init__(self):
        super().__init__()
        self.queue = []

    def get_first(self):
        try:
            return self.queue.pop()
        except Exception:
            return None

    def add_arrival(self, task):
        self.queue.append(task)


class WRR(BaseQueue):

    def __init__(self):
        super().__init__()
        self._priority_queues = [[] for i in range(3)]

    def get_first(self):
        for priority in range(3):
            if self._priority_queues[priority]:
                return self._priority_queues[priority].pop()
        return None

    def add_arrival(self, task):
        self._priority_queues[task.priority].append(task)


class NPPS(BaseQueue):
    def __init__(self):
        super().__init__()
        self.queue = []

    def get_first(self):
        try:
            return self.queue.pop()
        except Exception:
            return None

    def add_arrival(self, task):
        self.queue.append(task)
        self.queue = sorted(self.queue, key=lambda x: (x.priority, x.inter_arrival))


class Router:
    def __init__(self, PROCESSORS_NUM, SERVICE_POLICY, LENGTH_LIMIT, simulation_time):
        self.PROCESSORS_NUM = PROCESSORS_NUM
        self.SERVICE_POLICY = SERVICE_POLICY
        self.LENGTH_LIMIT = LENGTH_LIMIT  # TODO
        self.simulation_time = simulation_time

    def execute_all_tasks(self, all_tasks):
        current_time = 0
        for task in all_tasks:
            current_time += task.inter_arrival
            if current_time > self.simulation_time:
                break

        print('Simulation ended')

    def execute(self, task, ):
        self.SERVICE_POLICY.get_first()



In [8]:
X = 5 # parameter of the poisson distribution (in packet generation)
Y = 6 # parameter of the exponential distribution (in router - for service time generation)
T = 50 # Total simulation time

In [9]:
PROCESSORS_NUM = 2 # It can varry
SERVICE_POLICY = [FIFO(), WRR(), NPPS()][0]  # It can vary
LENGTH_LIMIT = 20 # It can varry

## Generate Packets
Here we are going to generate the packets ~ Poisson(X)

In [13]:
generator = np.random.default_rng()
packet_arrivals = generator.poisson(lam=X, size=T * X)
priorities = np.random.choice([0, 1, 2], p=[0.2, 0.3, 0.5], size = T * X)

## Generate Service Times
Here we are going to generate the service times ~ Exponential(Y)

In [11]:
generator = np.random.default_rng()
packet_times = generator.exponential(Y, size=T * X)

In [15]:
packets = [Task(inter_arrival=arrival, priority=priority, execute_time=time)
for (arrival, priority, time) in zip(packet_arrivals, priorities, packet_times)]


In [16]:
print(len(packets))


250


## Start Simulation


In [ ]:
# T : Total Time of Simlation
for t in T:
  # TODO

## Visualization
Here we are going to visualize the outputs and check the performance of our simulation.

### Mean Length of the Queue

### Mean time wasted in all Queues

### Mean time wasted in each Queue

### Mean Utilization of each processor

### Number of dropped packages

### How can we increase the utilization of the system?

Answer:

### CDF Plots of high-priority Packages

### Which of the Queuing Poilicies (3 policies) is better regarding the "number of dropped packages" and "number of packages received by the Host"?

Answer: